Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.359201
Minibatch accuracy: 12.5%
Validation accuracy: 9.4%
Minibatch loss at step 50: 1.637075
Minibatch accuracy: 37.5%
Validation accuracy: 48.9%
Minibatch loss at step 100: 0.945852
Minibatch accuracy: 62.5%
Validation accuracy: 68.7%
Minibatch loss at step 150: 0.362923
Minibatch accuracy: 87.5%
Validation accuracy: 76.0%
Minibatch loss at step 200: 0.801311
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 250: 1.134007
Minibatch accuracy: 68.8%
Validation accuracy: 77.3%
Minibatch loss at step 300: 0.393165
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.541325
Minibatch accuracy: 93.8%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.227111
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 450: 0.843726
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.701028
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

In [7]:
# Max pooling introduced in the function 'model'

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME') # max pooling
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME') # max pooling
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

In [ ]:
# Inception module: 1x1 conv, 1x1 conv with 3x3 conv, 1x1 conv with 5x5 conv, and 3x3 pool with 1x1 conv, in parallel.
# Using dropout + max norm regularization, linear learning rate decay, and AdamOptimizer.
# Works, but too resource intensive for my computer.

num_steps=1001

graph = tf.Graph()
with graph.as_default():
    depth = 16
    clip_norm = 1
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(1e-4, global_step, num_steps, 0.01)

    keep_prob=tf.placeholder(tf.float32)
    tf_x = tf.placeholder(tf.float32, shape=(None, image_size, image_size, num_channels))
    tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

    w1a_conv1x1 = tf.Variable(tf.truncated_normal([1, 1, num_channels, depth], stddev=0.1))
    b1a_conv1x1 = tf.Variable(tf.constant(0.1, shape=[depth]))
    h1a_conv1x1 = tf.nn.relu(tf.nn.conv2d(tf_x, w1a_conv1x1, [1, 1, 1, 1], 'SAME') + b1a_conv1x1)
    
    w1b_conv1x1 = tf.Variable(tf.truncated_normal([1, 1, num_channels, num_channels], stddev=0.1))
    b1b_conv1x1 = tf.Variable(tf.constant(0.1, shape=[num_channels]))
    h1b_conv1x1 = tf.nn.relu(tf.nn.conv2d(tf_x, w1b_conv1x1, [1, 1, 1, 1], 'SAME') + b1b_conv1x1)
    w1b_conv3x3 = tf.Variable(tf.truncated_normal([3, 3, num_channels, depth], stddev=0.1))
    b1b_conv3x3 = tf.Variable(tf.constant(0.1, shape=[depth]))
    h1b_conv3x3 = tf.nn.relu(tf.nn.conv2d(tf_x, w1b_conv3x3, [1, 1, 1, 1], 'SAME') + b1b_conv3x3)

    w1c_conv1x1 = tf.Variable(tf.truncated_normal([1, 1, num_channels, num_channels], stddev=0.1))
    b1c_conv1x1 = tf.Variable(tf.constant(0.1, shape=[num_channels]))
    h1c_conv1x1 = tf.nn.relu(tf.nn.conv2d(tf_x, w1c_conv1x1, [1, 1, 1, 1], 'SAME') + b1c_conv1x1)
    w1c_conv5x5 = tf.Variable(tf.truncated_normal([5, 5, num_channels, depth], stddev=0.1))
    b1c_conv5x5 = tf.Variable(tf.constant(0.1, shape=[depth]))
    h1c_conv5x5 = tf.nn.relu(tf.nn.conv2d(tf_x, w1c_conv5x5, [1, 1, 1, 1], 'SAME') + b1c_conv5x5)
    
    w1d_pool3x3 = tf.nn.max_pool(tf_x, [1, 3, 3, 1], [1, 3, 3, 1], 'SAME')
    w1d_conv1x1 = tf.Variable(tf.truncated_normal([1, 1, num_channels, depth], stddev=0.1))
    b1d_conv1x1 = tf.Variable(tf.constant(0.1, shape=[depth]))
    h1d_conv1x1 = tf.nn.relu(tf.nn.conv2d(w1d_pool3x3, w1d_conv1x1, [1, 1, 1, 1], 'SAME') + b1d_conv1x1)
    
    h1a_conv1x1_flat = tf.reshape(h1a_conv1x1, [-1, image_size * image_size * depth])
    h1b_conv3x3_flat = tf.reshape(h1b_conv3x3, [-1, image_size * image_size * depth])
    h1c_conv5x5_flat = tf.reshape(h1c_conv5x5, [-1, image_size * image_size * depth])
    h1d_conv1x1_flat = tf.reshape(h1d_conv1x1, [-1, (image_size // 3 + 1) * (image_size // 3 + 1) * depth])
    
    h1_flat = tf.nn.dropout(tf.concat_v2([h1a_conv1x1_flat, h1b_conv3x3_flat, h1c_conv5x5_flat, h1d_conv1x1_flat], 1), keep_prob)
    
    w2_fc = tf.Variable(tf.truncated_normal([h1_flat.get_shape().as_list()[1], 64], stddev=0.1))
    b2_fc = tf.Variable(tf.constant(0.1, shape=[64]))
    h2_fc = tf.nn.dropout(tf.nn.relu(tf.matmul(h1_flat, w2_fc) + b2_fc), keep_prob)

    w3_fc = tf.Variable(tf.truncated_normal([64, num_labels], stddev=0.1))
    b3_fc = tf.Variable(tf.constant(0.1, shape=[num_labels]))
    tf_y = tf.nn.relu(tf.matmul(h2_fc, w3_fc) + b3_fc)

    with tf.control_dependencies([w2_fc.assign(tf.clip_by_norm(w2_fc, clip_norm, axes=1)),
                                  w3_fc.assign(tf.clip_by_norm(w3_fc, clip_norm, axes=1))]):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf_y, tf_labels))
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
        tf_predictions = tf.nn.softmax(tf_y)

with tf.Session(graph=graph) as session:
    num_steps = 1001
    batch_size = 16
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        if step % (train_labels.shape[0] // batch_size) == 0:
            p = np.random.permutation(train_labels.shape[0])
            train_dataset=train_dataset[p]
            train_labels=train_labels[p]

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        _, l, predictions, lr = session.run([optimizer, loss, tf_predictions, learning_rate], feed_dict = {tf_x: batch_data, tf_labels: batch_labels, keep_prob: 0.8})

        if (step % 50 == 0):
            print("Learning rate at step %d: %f" % (step, lr))
            print("Minibatch loss: %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x: valid_dataset, keep_prob: 1.0}), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(tf_predictions.eval(feed_dict = {tf_x:test_dataset, keep_prob: 1.0}), test_labels))